In [2]:
 input_data='s3://poc-mach-learning/autogluon_sm/data/processed_payplan_data.csv'
 input_data.split("/")

['s3:',
 '',
 'poc-mach-learning',
 'autogluon_sm',
 'data',
 'processed_payplan_data.csv']

In [2]:
script_content = """
    """
    # This Lambda function deploys the model to SageMaker Endpoint. 
    # If Endpoint exists, then Endpoint will be updated with new Endpoint Config.
    """

    import json
    import boto3
    import time


    sm_client = boto3.client("sagemaker")


    def lambda_handler(event, context):

        print(f"Received Event: {event}")

        current_time = time.strftime("%m-%d-%H-%M-%S", time.localtime())
        endpoint_instance_type = event["endpoint_instance_type"]
        model_name = event["model_name"]
        endpoint_config_name = "{}-{}".format(event["endpoint_config_name"], current_time)
        endpoint_name = event["endpoint_name"]

        # Create Endpoint Configuration
        create_endpoint_config_response = sm_client.create_endpoint_config(
            EndpointConfigName=endpoint_config_name,
            ProductionVariants=[
                {
                    "InstanceType": endpoint_instance_type,
                    "InitialVariantWeight": 1,
                    "InitialInstanceCount": 1,
                    "VariantName": "AllTraffic",
                }
            ],
        )
        print(f"create_endpoint_config_response: {create_endpoint_config_response}")

        # Check if an endpoint exists. If no - Create new endpoint, if yes - Update existing endpoint
        list_endpoints_response = sm_client.list_endpoints(
            SortBy="CreationTime",
            SortOrder="Descending",
            NameContains=endpoint_name,
        )
        print(f"list_endpoints_response: {list_endpoints_response}")

        if len(list_endpoints_response["Endpoints"]) > 0:
            print("Updating Endpoint with new Endpoint Configuration")
            update_endpoint_response = sm_client.update_endpoint(
                EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
            )
            print(f"update_endpoint_response: {update_endpoint_response}")
        else:
            print("Creating Endpoint")
            create_endpoint_response = sm_client.create_endpoint(
                EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
            )
            print(f"create_endpoint_response: {create_endpoint_response}")

        return {"statusCode": 200, "body": json.dumps("Endpoint Created Successfully")}
    """

    with open('deploy_model_lambda.py', 'w') as f:
        f.write(script_content)


IndentationError: unexpected indent (<ipython-input-2-a2f16fd92f7b>, line 5)

In [3]:
import boto3

# Create a SageMaker client
sagemaker = boto3.client('sagemaker')

# List all endpoints
response = sagemaker.list_endpoints(
    SortBy='CreationTime',
    SortOrder='Descending'
)

# Print the names and statuses of the endpoints
for endpoint in response['Endpoints']:
    print('Name: {}'.format(endpoint['EndpointName']))
    print('Status: {}'.format(endpoint['EndpointStatus']))
    print('Creation time: {}'.format(endpoint['CreationTime']))
    print('Last modified time: {}'.format(endpoint['LastModifiedTime']))
    print('---')


Name: tf2-california-housing-endpoint-111-111-111
Status: InService
Creation time: 2023-05-16 15:24:38.879000+00:00
Last modified time: 2023-05-16 15:27:23.848000+00:00
---


In [6]:
import boto3

# Create a SageMaker client
sagemaker = boto3.client('sagemaker')

# List all endpoints
response = sagemaker.list_endpoints(
    SortBy='CreationTime',
    SortOrder='Descending'
)

# Delete all endpoints
for endpoint in response['Endpoints']:
    print('Deleting endpoint: {}'.format(endpoint['EndpointName']))
    sagemaker.delete_endpoint(
        EndpointName=endpoint['EndpointName']
    )
    print(f"Endpoint {endpoint['EndpointName']} deleted.")


Deleting endpoint: tf2-california-housing-endpoint-111-111-111
Endpoint tf2-california-housing-endpoint-111-111-111 deleted.
